In [ ]:
import pandas as pd
import tushare as ts
from tqdm import tqdm
import time
import numpy as np

# 设置TuShare的API token
ts.set_token('...')
pro = ts.pro_api()

# 读取指定范围的股票代码
file_path = r'...房地产A股\A股地产板块.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    specified_stocks = file.read().splitlines()

# 定义一个函数下载财务指标数据并检查是否为空
def download_fina_indicator(ts_code, start_date, end_date):
    fields = 'ts_code,ann_date,end_date,assets_turn'
    try:
        df = pro.query('fina_indicator', ts_code=ts_code, start_date=start_date, end_date=end_date, fields=fields)
        if df.empty:
            print(f"No data for {ts_code} in fina_indicator")
        return df
    except Exception as e:
        print(f"Error fetching data for {ts_code} in fina_indicator: {e}")
        return pd.DataFrame()

# 定义一个函数下载资产负债表数据并检查是否为空
def download_balancesheet(ts_code, start_date, end_date, retries=3):
    fields = 'ts_code,ann_date,end_date,total_cur_assets,total_hldr_eqy_inc_min_int,total_liab_hldr_eqy,cip,cip_total,undistr_porfit,total_assets'
    for attempt in range(retries):
        try:
            df = pro.query('balancesheet', ts_code=ts_code, start_date=start_date, end_date=end_date, fields=fields)
            if df.empty:
                print(f"No data for {ts_code} in balancesheet")
            return df
        except Exception as e:
            print(f"Error fetching data for {ts_code} in balancesheet (attempt {attempt + 1}): {e}")
            time.sleep(5)  # 等待5秒后重试
    return pd.DataFrame()

# 定义一个函数下载利润表数据并检查是否为空
def download_income(ts_code, start_date, end_date):
    fields = 'ts_code,ann_date,end_date,total_cogs,oper_cost,int_exp,comm_exp,biz_tax_surchg,admin_exp,fin_exp,total_revenue,non_oper_income'
    try:
        df = pro.query('income', ts_code=ts_code, start_date=start_date, end_date=end_date, fields=fields)
        if df.empty:
            print(f"No data for {ts_code} in income")
        return df
    except Exception as e:
        print(f"Error fetching data for {ts_code} in income: {e}")
        return pd.DataFrame()

# 设置报告期范围
start_date = '20170101'
end_date = '20240630'

# 收集所有股票的数据
all_stock_data = pd.DataFrame()

for code in tqdm(specified_stocks, desc="下载进度"):
    stock_data_fina = download_fina_indicator(code, start_date, end_date)
    stock_data_balance = download_balancesheet(code, start_date, end_date)
    stock_data_income = download_income(code, start_date, end_date)
    
    # 检查数据帧是否包含必要的列
    for df, name in zip([stock_data_fina, stock_data_balance, stock_data_income], 
                        ['fina_indicator', 'balancesheet', 'income']):
        if 'ts_code' not in df.columns:
            print(f"DataFrame from {name} is missing 'ts_code'")
        if 'ann_date' not in df.columns:
            print(f"DataFrame from {name} is missing 'ann_date'")
        if 'end_date' not in df.columns:
            print(f"DataFrame from {name} is missing 'end_date'")
    
    # 合并数据
    if not stock_data_fina.empty and not stock_data_balance.empty:
        stock_data = pd.merge(stock_data_fina, stock_data_balance, on=["ts_code", "ann_date", "end_date"], how="outer")
    else:
        stock_data = pd.DataFrame()
    
    if not stock_data.empty and not stock_data_income.empty:
        stock_data = pd.merge(stock_data, stock_data_income, on=["ts_code", "ann_date", "end_date"], how="outer")
    
    if not stock_data.empty:
        all_stock_data = pd.concat([all_stock_data, stock_data], ignore_index=True)
    time.sleep(60.0 / 60)  # 避免请求频率过高

# 过滤出end_date以1231结尾的数据，并填补空白数据
def fill_missing_year_end_data(df):
    df['year'] = df['end_date'].str[:4]
    result = pd.DataFrame()

    for code in df['ts_code'].unique():
        stock_df = df[df['ts_code'] == code]
        for year in stock_df['year'].unique():
            year_end_data = stock_df[(stock_df['end_date'] == f'{year}1231')]
            if year_end_data.empty:
                mid_year_data_1 = stock_df[(stock_df['end_date'] == f'{year}0630')]
                mid_year_data_2 = stock_df[(stock_df['end_date'] == f'{year}0930')]
                if not mid_year_data_1.empty:
                    result = pd.concat([result, mid_year_data_1])
                elif not mid_year_data_2.empty:
                    result = pd.concat([result, mid_year_data_2])
            else:
                result = pd.concat([result, year_end_data])

    return result

filtered_stock_data = fill_missing_year_end_data(all_stock_data)

# 删除重复的行，只保留每个ts_code每个end_date中的第一条记录
filtered_stock_data = filtered_stock_data.drop_duplicates(subset=['ts_code', 'end_date'], keep='first')

# 删除临时列
filtered_stock_data = filtered_stock_data.drop(columns=['year'])

# 保存到Excel
filtered_stock_data.to_excel('A股房地产板块.xlsx', index=False)
